In [1]:
!pip install -q pdfplumber datasets tqdm tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 113.5 MB/s eta 0:00:00


In [2]:
pdf_file  = "/content/LLM Engineers Handbook_ Master the art of engineering large -- Paul Iusztin, Maxime Labonne -- Expert Insight, 1, 2024 -- Packt Publishing -- c573723e1d4a903ff4efcb098037e804 -- Anna’s Archive.pdf"

In [3]:
import pdfplumber
import re
from tqdm import tqdm
import os

In [4]:
pdf_file = "/content/LLM Engineers Handbook_ Master the art of engineering large -- Paul Iusztin, Maxime Labonne -- Expert Insight, 1, 2024 -- Packt Publishing -- c573723e1d4a903ff4efcb098037e804 -- Anna’s Archive.pdf"
# Function to extract text from a single PDF
def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in tqdm(pdf.pages, desc="Extracting pages"):
            text = page.extract_text() or ""
            all_text += text + "\n\n"
    return all_text

# Run extraction
text = extract_text_from_pdf(pdf_file)
print("Text extraction complete!")
print("Total characters:", len(text))


Extracting pages: 100%|██████████| 523/523 [01:13<00:00,  7.11it/s]

Text extraction complete!
Total characters: 870990


In [5]:
def clean_text(text):
    text = text.replace("\u00A0", " ")
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"Page\s*\d+", "", text)
    return text.strip()

cleaned_text = clean_text(text)
print("Cleaned text length:", len(cleaned_text))

Cleaned text length: 870456


In [6]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

def split_into_chunks(text, max_tokens=1024):
    tokens = enc.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = enc.decode(tokens[i:i+max_tokens])
        chunks.append(chunk.strip())
    return chunks

chunks = split_into_chunks(cleaned_text, max_tokens=1024)
print(f"Created {len(chunks)} chunks (each ~1024 tokens)")

Created 187 chunks (each ~1024 tokens)


In [7]:
import json
import uuid

DATASET_PATH = "/content/book_dataset.jsonl"

with open(DATASET_PATH, "w", encoding="utf-8") as f:
    for chunk in chunks:
        record = {"id": str(uuid.uuid4()), "text": chunk}
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"Saved dataset → {DATASET_PATH}")

Saved dataset → /content/book_dataset.jsonl


In [8]:
from datasets import Dataset, DatasetDict

split = int(0.95 * len(chunks))
train_ds = Dataset.from_list([{"text": t} for t in chunks[:split]])
test_ds = Dataset.from_list([{"text": t} for t in chunks[split:]])
dataset = DatasetDict({"train": train_ds, "test": test_ds})
dataset.save_to_disk("/content/book_dataset_arrow")

print("Hugging Face Dataset created and saved to disk!")
print(dataset)

Saving the dataset (0/1 shards):   0%|          | 0/177 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Hugging Face Dataset created and saved to disk!
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 177
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10
    })
})


In [9]:
# show Dataset

from datasets import load_from_disk

# Path where you saved the dataset
ARROW_PATH = "/content/book_dataset_arrow"

# Load dataset
dataset = load_from_disk(ARROW_PATH)

# Display structure
print(dataset)
print("Columns:", dataset["train"].column_names)
print("Train size:", len(dataset["train"]))
print("Test size:", len(dataset["test"]))

# Preview samples
for i in range(2):
    print(f"\n--- TRAIN SAMPLE #{i} ---\n{dataset['train'][i]['text'][:500]}")


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 177
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10
    })
})
Columns: ['text']
Train size: 177
Test size: 10

--- TRAIN SAMPLE #0 ---
LLM Engineer’s Handbook Master the art of engineering large language models from concept to production Paul Iusztin Maxime Labonne LLM Engineer’s Handbook Copyright © 2024 Packt Publishing All rights reserved. No part of this book may be reproduced, stored in a retrieval system, or transmitted in any form or by any means, without the prior written permission of the publisher, except in the case of brief quotations embedded in critical articles or reviews. Every effort has been made in the prepar

--- TRAIN SAMPLE #1 ---
, Computer Vision and MLOps solutions. His latest contribution was at Metaphysic, where he served as one of their core engineers in taking large neural networks to production. He previously worked at CoreAI, Everseen, and Continen

In [33]:
# 1. Uninstall conflicting pyarrow versions
!pip uninstall -y pyarrow cudf-cu12 pylibcudf-cu12

# 2. Install compatible versions manually
!pip install -q "pyarrow>=22.0.0" "datasets>=3.0.0"

# 3. Now install Unsloth + related libraries
!pip install -q "unsloth>=2025.1.0" "trl>=0.9.6" \
"transformers>=4.44.0" "accelerate>=0.33.0" \
"peft>=0.12.0" bitsandbytes datasets

Found existing installation: pyarrow 22.0.0
Uninstalling pyarrow-22.0.0:
  Successfully uninstalled pyarrow-22.0.0


In [34]:
import os, torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

In [35]:
print("Torch version:", torch.__version__)
print(" CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(" GPU:", torch.cuda.get_device_name(0))

Torch version: 2.8.0+cu126
 CUDA available: True
 GPU: Tesla T4


In [38]:
from datasets import load_from_disk
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

In [39]:
# ==== MODEL SETTINGS ====
base_model = "meta-llama/Meta-Llama-3.1-8B"
max_seq_length = 2048

# ==== LoRA SETTINGS ====
lora_rank = 32
lora_alpha = 32
lora_dropout = 0
target_modules = ["q_proj","k_proj","v_proj","up_proj","down_proj","o_proj","gate_proj"]

# ==== DATASET SETTINGS ====
dataset_path = "/content/book_dataset_arrow"
dataset_text_field = "text"

# ==== TRAINING SETTINGS ====
num_epochs = 3
batch_size = 2
grad_accum = 8
learning_rate = 3e-4
weight_decay = 0.01
warmup_steps = 10
logging_steps = 10
seed = 0
report_to = "none"

# ==== OUTPUT SETTINGS ====
output_dir = "/content/drive/MyDrive/book_model/output"
save_merged = True

In [40]:
print(" Loading dataset from:", dataset_path)
dataset = load_from_disk(dataset_path)
print(dataset)
print("Columns:", dataset["train"].column_names)


 Loading dataset from: /content/book_dataset_arrow
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 177
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10
    })
})
Columns: ['text']


In [41]:
print("Loading base model:", base_model)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model,
    max_seq_length = max_seq_length,
    load_in_4bit = True,   # QLoRA mode
)
print("Model and tokenizer loaded.")


Loading base model: meta-llama/Meta-Llama-3.1-8B
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded.


In [42]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    target_modules = target_modules,
)
print("LoRA setup complete.")


LoRA setup complete.


In [43]:
fp16 = not is_bfloat16_supported()
bf16 = is_bfloat16_supported()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        learning_rate = learning_rate,
        lr_scheduler_type = "linear",
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = grad_accum,
        num_train_epochs = num_epochs,
        fp16 = fp16,
        bf16 = bf16,
        logging_steps = logging_steps,
        optim = "adamw_8bit",
        weight_decay = weight_decay,
        warmup_steps = warmup_steps,
        output_dir = output_dir,
        report_to = report_to,
        seed = seed,
    ),
)
print("Trainer initialized.")


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/177 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/10 [00:00<?, ? examples/s]

Trainer initialized.


In [44]:
trainer.train()
print("Training completed successfully!")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 177 | Num Epochs = 3 | Total steps = 36
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.929500
20,1.741400
30,1.578000


comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


Training completed successfully!


In [45]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

prompt = """Below is an instruction related to the book.
Write a response that appropriately completes the request.

### Instruction:
Summarize the main topic of this book in 3 bullet points.
### Response:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=streamer, max_new_tokens=256, use_cache=True)

<|begin_of_text|>Below is an instruction related to the book.
Write a response that appropriately completes the request.

### Instruction:
Summarize the main topic of this book in 3 bullet points.
### Response:
This book will show you how to: 1. Build a production-ready LLM Twin application with fine-tuned LLMs, RAG, and a prompt monitoring system. 2. Deploy the LLM Twin to AWS SageMaker and AWS ECR. 3. Build a data pipeline to collect, clean, and chunk data to train a fine-tuned LLM. The book will also teach you how to: 1. Create a custom LLM Twin architecture based on your application’s requirements. 2. Implement the LLM Twin’s core components, such as the RAG feature pipeline, the inference pipeline, and the prompt monitoring system. 3. Fine-tune LLMs using preference alignment. 4. Deploy the LLM Twin to AWS SageMaker and AWS ECR. 5. Build a data pipeline to collect, clean, and chunk data to train a fine-tuned LLM. Chapter 2 15 The book will also provide you with a clear understandi

In [46]:
from transformers import TextStreamer
import torch

prompt = """Below is an instruction related to the book.
Write a response that appropriately completes the request.

### Instruction:
What is the moral lesson of this book?
### Response:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)

# 🔧 Increased max_new_tokens + added eos_token_id
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=600,       # increase generation limit
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,
    )

print("\nCompleted text generation without cutoff.")


<|begin_of_text|>Below is an instruction related to the book.
Write a response that appropriately completes the request.

### Instruction:
What is the moral lesson of this book?
### Response:
The moral lesson of the book is that success in life is not determined by luck or external factors, but by hard work, perseverance, and the ability to adapt to changing circumstances. The story of Rags and Bones emphasizes the importance of taking responsibility for one's actions and making the most of the opportunities that come one's way. It also highlights the value of collaboration and the importance of building strong relationships with others. Ultimately, the book teaches readers that success is within their control and that they have the power to shape their own destiny.
The book also emphasizes the importance of empathy and understanding, and the power of human connection. Through the characters' experiences, readers are reminded that everyone has their own struggles and that we are all co

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [51]:
if save_merged:
    print("Saving merged model (16-bit)...")
    model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")
    print("Model saved locally at: /content/model")


Saving merged model (16-bit)...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 30671.33it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit:  50%|█████     | 2/4 [04:34<04:34, 137.24s/it]


KeyboardInterrupt: 

In [50]:
import shutil

model_path = "/content/model"
zip_path = "/content/drive/MyDrive/book_model/book_model.zip"

print("Zipping model folder...")
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', model_path)
print(f"Model zipped at: {zip_path}")


Zipping model folder...


KeyboardInterrupt: 